**Quantidade de leitos de urgência por município, dados do DataSUS**

Libraries

In [0]:
import pandas as pd
import numpy as np

###Leitura de CSV's

Leitos de repouso por município

In [2]:
leitos_repouso_ambulatorio = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/leitos_repouso_ambulatorio.csv", sep = ";", encoding = "iso-8859-1")
leitos_repouso_ambulatorio.columns = ["city","leitos_repouso_pediatras","leitos_repouso_feminino","leitos_repouso_masculino","leitos_repouso_indiferentes"]
leitos_repouso_ambulatorio.head(2)

,city,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes
0,110001 Alta Floresta D'Oeste,-,-,-,-
1,110037 Alto Alegre dos Parecis,4,4,4,-


In [3]:
leitos_repouso_urgencia = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/leitos_repouso_urgencia.csv", sep = ";", encoding = "iso-8859-1")
leitos_repouso_urgencia.columns = ["city","leitos_repouso_pediatras","leitos_repouso_feminino","leitos_repouso_masculino","leitos_repouso_indiferentes"]
leitos_repouso_urgencia.head(2)

,city,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes
0,110001 Alta Floresta D'Oeste,-,-,-,-
1,110037 Alto Alegre dos Parecis,-,-,-,1


Dados de Lagitude e Longitude dos municípios 

In [4]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/lat_long_municipios.csv")
city_lat_long.drop(columns = ["capital","Unnamed: 0"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [5]:
data_ibge = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/dados_ibge.csv")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município","Unnamed: 0"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação de dataframes

Juntando a tabela da localização geográfica com a dos dados do ibge

In [6]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Usando Split para retirar o código do município

In [7]:
new = leitos_repouso_urgencia["city"].str.split(" ", n = 1, expand = True)
leitos_repouso_urgencia["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
leitos_repouso_urgencia.drop(columns=["city"], inplace = True)
leitos_repouso_urgencia.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge
0,-,-,-,-,110001
1,-,-,-,1,110037


In [8]:
new = leitos_repouso_ambulatorio["city"].str.split(" ", n = 1, expand = True)
leitos_repouso_ambulatorio["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
leitos_repouso_ambulatorio.drop(columns=["city"], inplace = True)
leitos_repouso_ambulatorio.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge
0,-,-,-,-,110001
1,4,4,4,-,110037


Retirando o último digito para manter a igualdade, já que os dados da datasus vieram faltando o útltimo dígito

In [9]:
city_lat_long["codigo_ibge"] = city_lat_long["codigo_ibge"].map(lambda x: str(x)[:-1])
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,520005,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,310010,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge com a localização 

In [10]:
leitos_repouso_urgencia = leitos_repouso_urgencia.merge(city_lat_long, on = ["codigo_ibge"], sort = "city", how = "outer")
leitos_repouso_urgencia.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,-,-,-,-,000000,NaN,NaN,NaN,NaN,NaN
1,-,-,-,-,110000,NaN,NaN,NaN,NaN,NaN


In [11]:
leitos_repouso_ambulatorio = leitos_repouso_ambulatorio.merge(city_lat_long, on = ["codigo_ibge"], sort = "city", how = "outer")
leitos_repouso_ambulatorio.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,-,-,-,-,000000,NaN,NaN,NaN,NaN,NaN
1,-,-,-,-,110000,NaN,NaN,NaN,NaN,NaN


Apagando linha onde tem nulo em state

In [0]:
leitos_repouso_urgencia.dropna(subset=["state"], inplace = True)
leitos_repouso_ambulatorio.dropna(subset=["state"], inplace = True)

Retirando valores nulos

In [13]:
leitos_repouso_ambulatorio.replace("-", 0, inplace = True)
leitos_repouso_urgencia.replace("-", 0, inplace = True)
leitos_repouso_ambulatorio.replace(np.nan, 0, inplace = True)
leitos_repouso_urgencia.replace(np.nan, 0, inplace = True)
leitos_repouso_ambulatorio.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge,city,latitude,longitude,codigo_uf,state
2,0,0,0,0,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
3,5,0,0,12,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


Transformando colunas de string para inteiro

In [0]:
leitos_repouso_urgencia.iloc[:,1:4] = leitos_repouso_urgencia.iloc[:,:4].astype(int,copy = True)
leitos_repouso_ambulatorio.iloc[:,1:4] = leitos_repouso_ambulatorio.iloc[:,:4].astype(int,copy = True)

Somando leitos adultos

In [0]:
leitos_repouso_urgencia["leitos_repouso_adultos_total"] = leitos_repouso_urgencia.iloc[:,1:4].sum(axis=1)
leitos_repouso_ambulatorio["leitos_repouso_adultos_total"] = leitos_repouso_ambulatorio.iloc[:,1:4].sum(axis=1)

In [16]:
leitos_repouso_urgencia.head(2)

,leitos_repouso_pediatras,leitos_repouso_feminino,leitos_repouso_masculino,leitos_repouso_indiferentes,codigo_ibge,city,latitude,longitude,codigo_uf,state,leitos_repouso_adultos_total
2,0,0,0,0,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia,0
3,4,0,0,16,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia,16


###Download CSV

In [0]:
leitos_repouso_urgencia.to_csv("leitos_repouso_urgencia.csv")
leitos_repouso_ambulatorio.to_csv("leitos_repouso_ambulatorio.csv")